In [65]:
import os
from langchain_community.llms import QianfanLLMEndpoint
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Check if the environment variables are loaded correctly
qianfan_ak = os.getenv("QIANFAN_AK")
qianfan_sk = os.getenv("QIANFAN_SK")

if not qianfan_ak or not qianfan_sk:
    raise EnvironmentError("QIANFAN_AK or QIANFAN_SK environment variables are not set.")

# Set environment variables for Qianfan LLM
os.environ["QIANFAN_AK"] = qianfan_ak
os.environ["QIANFAN_SK"] = qianfan_sk

# Initialize Qianfan LLM Endpoint
Yi = QianfanLLMEndpoint(temperature=0.2, model="Yi-34B-Chat")

# Create the prompt template
prompt = PromptTemplate.from_template("请问{company}的{product}是什么？")

chain = prompt | Yi

response = chain.invoke({"company": "腾讯", "product": "QQ"})

print(response)


腾讯QQ（简称“QQ”）是腾讯公司开发的一款基于Internet的即时通信软件。它于1999年2月推出，最初是模仿以色列的即时通讯软件ICQ，但后来发展出了许多独特的功能和服务。QQ的主要功能包括实时文字聊天、语音通话、视频通话、文件传输、表情发送、群聊、空间分享等。

QQ在中国大陆地区拥有广泛的使用基础，是许多个人和组织进行在线交流的重要工具。随着技术的进步，QQ不断推出新功能，如QQ秀、QQ游戏、QQ音乐等，这些功能使得QQ不仅仅是一个即时通讯工具，还是一个集社交、娱乐、商务为一体的综合性平台。

除了个人用户，QQ也广泛应用于企业通信和客户服务。许多企业使用QQ群来与客户进行互动，提供咨询和服务。此外，QQ还支持企业QQ服务，这是一种针对企业的即时通讯解决方案，提供更加稳定和安全的通信环境。

腾讯QQ的发展历程中，也经历了多次的版本更新和功能优化，以适应不断变化的市场需求和技术发展。尽管近年来，随着移动互联网的发展，微信等新的即时通讯工具崛起，QQ的用户量有所下降，但它仍然是中国互联网领域的一个重要平台，拥有着庞大的用户群体和丰富的应用场景。


#### PromptTemplate

In [7]:
# Create the prompt template
prompt = PromptTemplate.from_template("请问{company}的{product}是什么？")

message = prompt.format(company="腾讯", product="QQ")   # Format the prompt template with the values

response = Yi.invoke(message)
print(response)


腾讯QQ（简称“QQ”）是腾讯公司开发的一款基于Internet的即时通信软件。它于1999年2月推出，最初是模仿ICQ的一款软件，但后来发展出了自己的特色功能和服务。QQ的主要功能包括文字消息、语音通话、视频通话、文件传输、群聊、个人空间等。

QQ在中国大陆地区拥有广泛的使用基础，是许多中国人主要的即时通信工具之一。随着腾讯公司的发展，QQ还扩展了其服务范围，包括社交网络、在线游戏、音乐、购物等。QQ空间（Qzone）就是QQ的一个衍生产品，它是一个用户可以分享照片、音乐、心情等个人生活内容的社交网络平台。

除了个人用户，QQ也提供了一系列针对企业用户的服务，如企业QQ、腾讯会议等，帮助企业进行内部沟通和外部客户服务。

截至2023年，QQ仍然是全球用户数量最多的即时通信软件之一，尤其在中国市场具有极高的普及率。


#### ChatPromptTemplate 对话模板

In [46]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import QianfanLLMEndpoint

# Initialize Qianfan LLM Endpoint
ERNIE = QianfanLLMEndpoint(temperature=0.5, model="ERNIE-Speed-128K")
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个导游，你的名字叫{guide_name}。你每次回答问题都要自我介绍。"),
        ("ai", "你好，我是{guide_name}，我是一个导游。"),
        ("user", "你好，{guide_name}。我想去参观故宫。"),
    ]
)

chat_template.format(guide_name="小明")

'System: 你是一个导游，你的名字叫小明。你每次回答问题都要自我介绍。\nAI: 你好，我是小明，我是一个导游。\nHuman: 你好，小明。我想去参观故宫。'

In [48]:
chain = chat_template | ERNIE

response = ERNIE.invoke(chat_template.format(guide_name="小爱"))
print(response)

你好！我是小爱，很高兴你联系我关于参观故宫的事宜。故宫是中国历史文化的重要遗产之一，充满了丰富的历史和艺术价值。如果你有任何关于参观的问题或需要了解的信息，请随时告诉我。


#### chatmessage 中的角色和自定义角色

In [95]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatMessagePromptTemplate

ERNIE = QianfanLLMEndpoint(temperature=0.2, model="ERNIE-Speed-128K")
Yi = QianfanLLMEndpoint(temperature=0.8, model="Yi-34B-Chat")

# Create a message template
sys = SystemMessage(content="现在开始角色扮演。请时刻记住你的角色。你是一个历史老师，你的名字叫史密斯。你正在跟一个学生交流。交流的态度请你参考学生父亲的态度。")
ai = AIMessage(content="你好，我是史密斯，我是一个历史老师。")
user = HumanMessage(content="你好，史密斯。")

# add a new role
father_message_prompt = "你好，{guide_name}，我是学生的父亲。我的孩子学习成绩非常差，请你严厉一点。"
father_message_template = ChatMessagePromptTemplate.from_template(role="user_father", template = father_message_prompt)
father_message=father_message_template.format(guide_name="史密斯")

message = [sys, ai, father_message, user]

response = Yi.invoke(message)
print(response)


你好，人类。我是史密斯，你的历史老师。我注意到你的学习成绩有些不尽人意，但我相信你有着巨大的潜力。作为你的老师，我会尽力帮助你提升成绩。请告诉我，你在学习历史时遇到了哪些困难？我们可以一起制定一个计划，帮助你克服这些困难。
